In [1]:
from datetime import datetime
from datetime import timedelta
import json
import re
from os import path, environ
from time import sleep
import os

import requests
#from geopy.geocoders import GoogleV3
from bs4 import BeautifulSoup
from dateutil.parser import parse

from selenium import webdriver      # imports
from selenium.webdriver.support.ui import Select

In [2]:
headers = {
    "User-Agent": "Mozilla/5.0 (Macintosh; U; Intel Mac OS X 10.5; en-US; rv:1.9.1b3) Gecko/20090305 Firefox/3.1b3 GTB5"
}
base_url = 'https://www.gunviolencearchive.org'

In [3]:
#These files need to be in the same directory as the jupyter notebook file.
SHOOTINGS_JSON = 'shootings.json'

SHOOTINGS_COMPACT_JSON = 'shootings_compact.json'

### Function declarations:

In [31]:

def scrape_results(current_url):
    headers = {
    "User-Agent": "Mozilla/5.0 (Macintosh; U; Intel Mac OS X 10.5; en-US; rv:1.9.1b3) Gecko/20090305 Firefox/3.1b3 GTB5"
}
    url = current_url
    res = requests.get(url, headers=headers)
    soup = BeautifulSoup(res.text, features="lxml")
    table = soup.find("table")
    headers = [
        re.sub(r"[^a-z]", "", th.get_text().lower()) for th in table.find_all("th")
    ]
    rows = table.find("tbody").find_all("tr")
    #print(rows)
    results = []
    for row in rows:
        try:
            cols = row.find_all("td")
            #print(cols)
            links = row.select(".links a")
            incident_id = int(re.sub(r"[^\d]", "", links[0].get("href")))
            if not incident_id > 0:
                raise RuntimeError(f"Invalid incident ID {incident_id} at {url}")
            results.append(
                {
                    "id": int(re.sub(r"[^\d]", "", links[0].get("href"))),
                    "date": parse(cols[headers.index("incidentdate")].get_text()),
                    "state": cols[headers.index("state")].get_text(),
                    "city": cols[headers.index("cityorcounty")].get_text(),
                    "address": cols[headers.index("address")].get_text(),
                    "killed": int(cols[headers.index("killed")].get_text()),
                    "injured": int(cols[headers.index("injured")].get_text()),
                    "source": links[1].get("href") if len(links) > 1 else None,
                }
            )
        except:
            continue
        
    return results

In [27]:
def merge_shooting_data(scraped_data):
    total_new = 0
    for shooting in scraped_data:
        if shooting["id"] not in keyed_shooting_data:
            print(f'New shooting found: {shooting["id"]}, {shooting["date"]}')
            keyed_shooting_data[shooting["id"]] = {}
            total_new += 1
        keyed_shooting_data[shooting["id"]].update(shooting)
    return total_new

In [36]:

def parse_page(next_url):
    # HTTP GET requests
    page = requests.get(next_url)

    # Checking if we successfully fetched the URL
    if page.status_code == requests.codes.ok:
        bs = BeautifulSoup(page.text, 'lxml')
        try:
            current_url = next_url
            print(current_url)
            results = scrape_results(current_url)
            # Check for empty result:
            if not results:
                return "empty"
            else:
                #print(results)
                merge_shooting_data(results)
                next_page_text = bs.find('ul', class_="pager").findAll('li')[-2].text
                if next_page_text == 'next ›':
                    next_page_partial = bs.find('ul', class_="pager").findAll('li')[-2].find('a')['href']
                    next_page_url = base_url + next_page_partial
                    current_url = next_page_url
                    print(next_page_url)
                    results = scrape_results(current_url)
                    merge_shooting_data(results)
                    sleep(3)
                    if next_page_text == 'next ›':
                        parse_page(next_page_url)
                        sleep(1)
                    else:
                        merge_shooting_data(results)
        except:
            current_url = next_url
            results = scrape_results(current_url)
            merge_shooting_data(results)
            

### Main implementation:

In [37]:
# Test
year = '2019'
month= 'Jan'
day = 1


driver = webdriver.Chrome('/Users/jasonlim/Desktop/EFGS/GVA-master/All shootings/Scripts/chromedriver')

                # open some page using get method       - url -- > parameters

driver.get('https://www.gunviolencearchive.org/query')



add_rule_bar2 = driver.find_element_by_class_name('filter-dropdown-trigger')
add_rule_bar2.click()
sleep(4)


# Choose Date

driver.find_element_by_link_text('Date').click()

# 1 seconds for page loading
sleep(1)

# From DATE
# pull out calender
from_date = driver.find_element_by_css_selector("input[id$='filter-field-date-from']")
from_date.click()
# month select
month1 = Select(driver.find_element_by_xpath('/html/body/div[4]/div[1]/div/table/thead/tr[1]/th[2]/select[1]'))
sleep(1)
month1.select_by_visible_text(month)
sleep(1)


# year select
year1 = Select(driver.find_element_by_xpath('/html/body/div[4]/div[1]/div/table/thead/tr[1]/th[2]/select[2]'))
sleep(1)
year1.select_by_visible_text(str(year))

sleep(1)


# start date

start_date = driver.find_elements_by_xpath("/html/body/div[4]/div[1]/div/table/tbody/tr/td[@class='available']")[day]
start_date.click()

# TO DATE
# pull out calender
# end date
end_date = driver.find_element_by_css_selector("input[id$='filter-field-date-to']")
end_date.click()

# month select
sleep(1)
month2= Select(driver.find_element_by_xpath('/html/body/div[5]/div[1]/div/table/thead/tr[1]/th[2]/select[1]'))
sleep(1)
month2.select_by_visible_text(month)
sleep(1)


# year select
year2 = Select(driver.find_element_by_xpath('/html/body/div[5]/div[1]/div/table/thead/tr[1]/th[2]/select[2]'))
sleep(1)
year2.select_by_visible_text(str(year))

sleep(1)

# End date
# Currently, I made up first available date, which 0 index from the list.
End_date = driver.find_elements_by_xpath("/html/body/div[5]/div[1]/div/table/tbody/tr/td[@class='available']")[day]
End_date.click()

sleep(1)

# Click add a rule
add_rule_bar = driver.find_element_by_class_name('filter-dropdown-trigger')
add_rule_bar.click()
sleep(1)

# Select 'Incident Characteristic'

driver.find_element_by_link_text('Incident Characteristic').click()

# select Accidental Shooting
sleep(1)
acc_shoot = Select(driver.find_element_by_xpath("/html/body/section/div/div/div/div[2]/div/form/div/div[3]/div[2]/div[2]/div[2]/div[2]/div/span/select"))
sleep(1)
acc_shoot.select_by_visible_text("Accidental Shooting")
# Click search
driver.find_element_by_css_selector("input[id$='edit-actions-execute']").click()

current_year = datetime.now().year
with open(SHOOTINGS_JSON) as json_file:
    shooting_data = json.load(json_file)
keyed_shooting_data = {}
for shooting in shooting_data:
    keyed_shooting_data[shooting["id"]] = shooting
    shooting["date"] = parse(shooting["date"])

print(f"Loading existing shooting data with {len(shooting_data)} incidents")
current_url = driver.current_url
parse_page(current_url)

Loading existing shooting data with 27 incidents
https://www.gunviolencearchive.org/query/4deebdc0-0869-4855-ba58-c74b065dcb50
empty


In [52]:

year = ['2014','2015', '2016','2017','2018','2019', '2020']
month = ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']


for years in range(len(year)):
    for months in range(len(month)):
        
        if year[years]=='2020' or year[years]=='2016':
            if month[months]=='Jan' or month[months]=='Mar' or month[months]=='May' or month[months]=='Jul' or month[months]=='Aug' or month[months]=='Oct' or month[months]=='Dec':
                day = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30]
            elif month[months]=='Feb':
                day = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28]
            else :
                day = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29]
        else:
            if month[months]=='Jan' or month[months]=='Mar' or month[months]=='May' or month[months]=='Jul' or month[months]=='Aug' or month[months]=='Oct' or month[months]=='Dec':
                day = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30]
            elif month[months]=='Feb':
                day = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27] 
            else :
                day = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29]
        for days in range(len(day)):
            
            c_date = '{}-{}-{}'.format(year[years],month[months],day[days]+1)
            c_date = datetime.strptime(c_date,"%Y-%b-%d")
            day_difference = (datetime.now() - c_date).days
            if (day_difference >3):
                driver = webdriver.Chrome('/Users/jasonlim/Desktop/EFGS/GVA-master/All shootings/Scripts/chromedriver')

                # open some page using get method       - url -- > parameters

                driver.get('https://www.gunviolencearchive.org/query')
                
                
                
                add_rule_bar2 = driver.find_element_by_class_name('filter-dropdown-trigger')
                add_rule_bar2.click()
                sleep(3)
                
                
                # Choose Date
                
                driver.find_element_by_link_text('Date').click()
                
                # 3 seconds for page loading
                sleep(3)
                
                # From DATE
                # pull out calender
                from_date = driver.find_element_by_css_selector("input[id$='filter-field-date-from']")
                from_date.click()
                # month select
                month1 = Select(driver.find_element_by_xpath('/html/body/div[4]/div[1]/div/table/thead/tr[1]/th[2]/select[1]'))
                sleep(3)
                month1.select_by_visible_text(month[months])
                sleep(3)
                
                
                # year select
                year1 = Select(driver.find_element_by_xpath('/html/body/div[4]/div[1]/div/table/thead/tr[1]/th[2]/select[2]'))
                sleep(3)
                year1.select_by_visible_text(str(year[years]))
                
                sleep(3)
                
                
                # start date
                
                start_date = driver.find_elements_by_xpath("/html/body/div[4]/div[1]/div/table/tbody/tr/td[@class='available']")[day[days]]
                start_date.click()
                
                # TO DATE
                # pull out calender
                # end date
                end_date = driver.find_element_by_css_selector("input[id$='filter-field-date-to']")
                end_date.click()
                
                # month select
                sleep(1)
                month2= Select(driver.find_element_by_xpath('/html/body/div[5]/div[1]/div/table/thead/tr[1]/th[2]/select[1]'))
                sleep(1)
                month2.select_by_visible_text(month[months])
                sleep(1)
                
                
                # year select
                year2 = Select(driver.find_element_by_xpath('/html/body/div[5]/div[1]/div/table/thead/tr[1]/th[2]/select[2]'))
                sleep(3)
                year2.select_by_visible_text(str(year[years]))
                
                sleep(3)
                
                # End date
                # Currently, I made up first available date, which 0 index from the list.
                End_date = driver.find_elements_by_xpath("/html/body/div[5]/div[1]/div/table/tbody/tr/td[@class='available']")[day[days]]
                End_date.click()
                
                sleep(3)
                
                # Click add a rule
                add_rule_bar = driver.find_element_by_class_name('filter-dropdown-trigger')
                add_rule_bar.click()
                sleep(3)
                
                # Select 'Incident Characteristic'
                
                driver.find_element_by_link_text('Incident Characteristic').click()
                # select Accidental Shooting
                sleep(3)
                acc_shoot = Select(driver.find_element_by_xpath("/html/body/section/div/div/div/div[2]/div/form/div/div[3]/div[2]/div[2]/div[2]/div[2]/div/span/select"))
                sleep(3)
                acc_shoot.select_by_visible_text("Accidental Shooting")
                
                # Click search
                driver.find_element_by_css_selector("input[id$='edit-actions-execute']").click()
                
                start_year = 2014
                current_year = datetime.now().year
                with open(SHOOTINGS_JSON) as json_file:
                    shooting_data = json.load(json_file)

                keyed_shooting_data = {}
                for shooting in shooting_data:
                    keyed_shooting_data[shooting["id"]] = shooting
                    shooting["date"] = parse(shooting["date"])

                print(f"Loading existing shooting data with {len(shooting_data)} incidents")
                current_url = driver.current_url
                if parse_page(current_url)=='empty':
                    continue
                else:
                    parse_page(current_url)
                    print("Sorting results")
                    updated_shooting_data = sorted(
                        keyed_shooting_data.values(), key=lambda row: row['date']
                    )
    
                    updated_shootings_compact = []
                    for shooting in updated_shooting_data:
                        shooting["date"] = shooting["date"].strftime("%Y-%m-%d")
                        updated_shootings_compact.append(
                            [
                                shooting["id"],
                                shooting["date"],
                                shooting["killed"],
                                shooting["injured"],
    
                            ]
                        )
    
    
                    print("Writing results")
                    with open(SHOOTINGS_JSON, "w") as outfile:
                        json.dump(updated_shooting_data, outfile, indent=2)
    
                    with open(SHOOTINGS_COMPACT_JSON, "w") as outfile:
                        json.dump(updated_shootings_compact, outfile, separators=(",", ":"))
    
                    print(f"Done! {len(updated_shooting_data)} shootings found")
                    driver.close()
            else:
                break
            


Loading existing shooting data with 1175 incidents
https://www.gunviolencearchive.org/query/ec1923c8-8a2c-4632-86d5-b2781bfba5cd
New shooting found: 1204113, 2018-09-01 00:00:00
New shooting found: 1202105, 2018-09-01 00:00:00
New shooting found: 1201973, 2018-09-01 00:00:00
New shooting found: 1201640, 2018-09-01 00:00:00
https://www.gunviolencearchive.org/query/ec1923c8-8a2c-4632-86d5-b2781bfba5cd
Sorting results
Writing results
Done! 1179 shootings found
Loading existing shooting data with 1179 incidents
https://www.gunviolencearchive.org/query/408a1658-6f1b-422b-a295-5fb66e0ebd7d
New shooting found: 1201905, 2018-09-02 00:00:00
New shooting found: 1563922, 2018-09-02 00:00:00
New shooting found: 1204130, 2018-09-02 00:00:00
New shooting found: 1202474, 2018-09-02 00:00:00
https://www.gunviolencearchive.org/query/408a1658-6f1b-422b-a295-5fb66e0ebd7d
Sorting results
Writing results
Done! 1183 shootings found
Loading existing shooting data with 1183 incidents
https://www.gunviolencear

Sorting results
Writing results
Done! 1241 shootings found
Loading existing shooting data with 1241 incidents
https://www.gunviolencearchive.org/query/b720984a-3a13-465c-ae71-fdb4fb5fc8f6
New shooting found: 1215198, 2018-09-20 00:00:00
New shooting found: 1214045, 2018-09-20 00:00:00
https://www.gunviolencearchive.org/query/b720984a-3a13-465c-ae71-fdb4fb5fc8f6
Sorting results
Writing results
Done! 1243 shootings found
Loading existing shooting data with 1243 incidents
https://www.gunviolencearchive.org/query/ac3e8d0d-a3cc-4ddb-8097-25f8c9177585
empty
https://www.gunviolencearchive.org/query/ac3e8d0d-a3cc-4ddb-8097-25f8c9177585
empty
Sorting results
Writing results
Done! 1243 shootings found
Loading existing shooting data with 1243 incidents
https://www.gunviolencearchive.org/query/fd6e56ed-3fcf-43c1-882d-05b9d5661439
New shooting found: 1215761, 2018-09-22 00:00:00
New shooting found: 1216831, 2018-09-22 00:00:00
New shooting found: 1216411, 2018-09-22 00:00:00
New shooting found: 121

https://www.gunviolencearchive.org/query/ed2277e5-7217-4d10-8a88-3bff9ffc4a4c
Sorting results
Writing results
Done! 1310 shootings found
Loading existing shooting data with 1310 incidents
https://www.gunviolencearchive.org/query/87b19f8d-7242-449c-8913-c58a7f21acfc
New shooting found: 1230055, 2018-10-09 00:00:00
New shooting found: 1229241, 2018-10-09 00:00:00
New shooting found: 1229019, 2018-10-09 00:00:00
New shooting found: 1230175, 2018-10-09 00:00:00
https://www.gunviolencearchive.org/query/87b19f8d-7242-449c-8913-c58a7f21acfc
Sorting results
Writing results
Done! 1314 shootings found
Loading existing shooting data with 1314 incidents
https://www.gunviolencearchive.org/query/b6ec2c3f-d730-4ff4-981f-a06935896ee0
New shooting found: 1230462, 2018-10-10 00:00:00
New shooting found: 1229736, 2018-10-10 00:00:00
New shooting found: 1229879, 2018-10-10 00:00:00
https://www.gunviolencearchive.org/query/b6ec2c3f-d730-4ff4-981f-a06935896ee0
Sorting results
Writing results
Done! 1317 shoo

https://www.gunviolencearchive.org/query/745f4b7e-e493-474e-bc5f-81813509441e
Sorting results
Writing results
Done! 1380 shootings found
Loading existing shooting data with 1380 incidents
https://www.gunviolencearchive.org/query/1ae8b4b5-4ef5-4bec-9e6f-41b1d2deaaea
New shooting found: 1242733, 2018-10-27 00:00:00
New shooting found: 1242201, 2018-10-27 00:00:00
New shooting found: 1243099, 2018-10-27 00:00:00
https://www.gunviolencearchive.org/query/1ae8b4b5-4ef5-4bec-9e6f-41b1d2deaaea
Sorting results
Writing results
Done! 1383 shootings found
Loading existing shooting data with 1383 incidents
https://www.gunviolencearchive.org/query/929580fb-5f1a-4dac-bb45-59225155fe96
New shooting found: 1242882, 2018-10-28 00:00:00
New shooting found: 1242783, 2018-10-28 00:00:00
New shooting found: 1242779, 2018-10-28 00:00:00
https://www.gunviolencearchive.org/query/929580fb-5f1a-4dac-bb45-59225155fe96
Sorting results
Writing results
Done! 1386 shootings found
Loading existing shooting data with 1

https://www.gunviolencearchive.org/query/990ae498-c8da-4790-ab60-4febf84678c7
Sorting results
Writing results
Done! 1451 shootings found
Loading existing shooting data with 1451 incidents
https://www.gunviolencearchive.org/query/91f6854f-a754-4827-8f73-b77a064f4016
New shooting found: 1255350, 2018-11-14 00:00:00
New shooting found: 1255672, 2018-11-14 00:00:00
New shooting found: 1255632, 2018-11-14 00:00:00
New shooting found: 1257251, 2018-11-14 00:00:00
New shooting found: 1256230, 2018-11-14 00:00:00
https://www.gunviolencearchive.org/query/91f6854f-a754-4827-8f73-b77a064f4016
Sorting results
Writing results
Done! 1456 shootings found
Loading existing shooting data with 1456 incidents
https://www.gunviolencearchive.org/query/c8e334f5-8674-4408-b8bd-6b964b28875a
New shooting found: 1256595, 2018-11-15 00:00:00
New shooting found: 1256271, 2018-11-15 00:00:00
New shooting found: 1256752, 2018-11-15 00:00:00
New shooting found: 1256731, 2018-11-15 00:00:00
New shooting found: 1260118

New shooting found: 1267109, 2018-11-29 00:00:00
New shooting found: 1266173, 2018-11-29 00:00:00
New shooting found: 1265612, 2018-11-29 00:00:00
New shooting found: 1266388, 2018-11-29 00:00:00
https://www.gunviolencearchive.org/query/a93e9987-4ceb-415e-a16d-64a99e70ad64
Sorting results
Writing results
Done! 1536 shootings found
Loading existing shooting data with 1536 incidents
https://www.gunviolencearchive.org/query/0c87a6a8-291a-48d2-a15c-4f1cb2237df0
New shooting found: 1266675, 2018-11-30 00:00:00
New shooting found: 1266905, 2018-11-30 00:00:00
New shooting found: 1316796, 2018-11-30 00:00:00
New shooting found: 1266841, 2018-11-30 00:00:00
New shooting found: 1269868, 2018-11-30 00:00:00
New shooting found: 1267534, 2018-11-30 00:00:00
New shooting found: 1268594, 2018-11-30 00:00:00
New shooting found: 1513163, 2018-11-30 00:00:00
https://www.gunviolencearchive.org/query/0c87a6a8-291a-48d2-a15c-4f1cb2237df0
Sorting results
Writing results
Done! 1544 shootings found
Loading e

New shooting found: 1278908, 2018-12-15 00:00:00
New shooting found: 1278045, 2018-12-15 00:00:00
New shooting found: 1279061, 2018-12-15 00:00:00
New shooting found: 1280781, 2018-12-15 00:00:00
New shooting found: 1278409, 2018-12-15 00:00:00
https://www.gunviolencearchive.org/query/91d79932-177a-422b-89b1-ee5fabc33fe7
Sorting results
Writing results
Done! 1618 shootings found
Loading existing shooting data with 1618 incidents
https://www.gunviolencearchive.org/query/fac9f77f-e659-4a89-b01c-0ac83c67cda5
New shooting found: 1278429, 2018-12-16 00:00:00
New shooting found: 1279112, 2018-12-16 00:00:00
New shooting found: 1283283, 2018-12-16 00:00:00
https://www.gunviolencearchive.org/query/fac9f77f-e659-4a89-b01c-0ac83c67cda5
Sorting results
Writing results
Done! 1621 shootings found
Loading existing shooting data with 1621 incidents
https://www.gunviolencearchive.org/query/19d20cb2-39e0-436e-b0f7-e1bebe1b59c4
New shooting found: 1280144, 2018-12-17 00:00:00
https://www.gunviolencearch